# OpenAI API 使用範例
## 教學目標
- 使用 Python 語法來透過 OpenAI API 呼叫 OpenAI 模型
- 首先需要到此取得你的 API https://platform.openai.com/api-keys
## 參考資料
- [OpenAI Cookbook](https://github.com/openai/openai-cookbook)
- [OpenAI Cookbook - How to format inputs_to ChatGPT models](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_format_inputs_to_ChatGPT_models.ipynb)
- [OpenAI API Models](https://platform.openai.com/docs/models)

## 安裝
透過 `pip` 來安裝 `openai` 函式庫

In [1]:
!pip install openai

In [2]:
from openai import OpenAI
my_api_key = "在這邊輸入你的 API"

In [3]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
import logging
logger = logging.getLogger('my_logger')

In [4]:
import os

# 建立 logs 資料夾來儲存 OpenAI 的輸出
os.mkdir("logs")
LOG_PATH = "logs/chatgpt.txt"

In [5]:
client = OpenAI(api_key=my_api_key)

## 在輸入時給予沒有給範例
- 僅單純給予 prompt

In [6]:
# Use tenacity to retry the API call
# 基本上 @retry 這行不會需要被改變
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(message: str, temperature: int):
    response = client.chat.completions.create(
        model="gpt-4", # 可選擇任意 OpenAI 的模型
        messages=[
            {
                "role": "system",
                # content 的內容可以自行根據任務性質來進行調整
                "content": "You are a professional NLP scientist who has published numerous research papers.",
            },
            {"role": "user", "content": message},
        ],
        temperature=temperature,
    )
    return response

In [7]:
message = "Please explain BERT for me like a five year old child."
response = completion_with_backoff(message, temperature=0)

In [8]:
# 觀察 response 的內容

for item in response:
    print(item)

('id', 'chatcmpl-8Ki1ZS9miOPQn6sHhass9Ahp5bzsJ')
('choices', [Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Sure, imagine you have a big box of toys, but they are all mixed up. You don\'t know which toy is which, and you can\'t play properly. Now, imagine you have a super toy-organizer robot. This robot doesn\'t just organize your toys based on their color or size, but it also understands which toys you like to play with together, like your toy cars with your toy garage, or your action figures with your toy castle. \n\nBERT is like that super toy-organizer robot, but for words and sentences. It doesn\'t just look at words on their own, but it also understands how words relate to each other in a sentence. It can understand that in the sentence "He went to the bank", the word "bank" is a place where money is kept, not the side of a river, because of the other words in the sentence. This helps BERT to understand language much better, and it can help us with 

In [9]:
output = response.choices[0].message.content
logging.info(output)
logging.info("=" * 30)

## 在輸入時給予少量範例
- 又稱做 few-shot 或 In-context Learning
- 本範例以 `role_play` 變數進行舉例
- `role_play` 中包含三組範例，分別有我希望 GPT 回答的輸出
- 如果希望 GPT 的輸出帶有一定格式，如 `role_play` 中我希望輸出有 《 》，則建議以此方式給予模型輸入。

In [10]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(message: str, temperature: int):

    role_play = (
        "你現在是一個維基百科搜索引擎，我給你的輸入是一段簡短的敘述，這段敘述可能是不正確的，請幫我判斷該敘述的主詞，並輸出與該主詞最接近的維基百科文章名稱。"
        + '範例1，輸入："天衛三軌道在天王星內部的磁層，以《 仲夏夜之夢 》作者緹坦妮雅命名。"'
        + '輸出："《天衛三》"。'
        + '範例2，輸入："信天翁科的活動範圍位於北冰洋以及南太平洋，牠的翼展可達到3.7米，是世界上現存的翼展最大的鳥類。"'
        + '輸出："《信天翁科》"。'
        + '範例3，輸入："南京大學附屬中學，從中國江蘇省遷移。"'
        + '輸出："《南大附中》"。'
    )

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": role_play,
            },
            {"role": "user", "content": message},
        ],
        temperature=temperature,
    )
    return response

In [11]:
message = "成功大學有12座籃球場。"
response = completion_with_backoff(message, temperature=0)

In [12]:
# 觀察 response 的內容

for item in response:
    print(item)

('id', 'chatcmpl-8Ki1tNEKJvtDub6PkrSFpDfOHQUe6')
('choices', [Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='"《成功大學》"', role='assistant', function_call=None, tool_calls=None))])
('created', 1699946501)
('model', 'gpt-4-0613')
('object', 'chat.completion')
('system_fingerprint', None)
('usage', CompletionUsage(completion_tokens=8, prompt_tokens=351, total_tokens=359))


In [13]:
output = response.choices[0].message.content

# Write outputs
with open(LOG_PATH, "a") as f:
    f.write(message + "\n")
    f.write(output + "\n")
    f.write("=" * 30 + "\n")